In [1]:
from data import constants
from data.eluc_data import ELUCData
from predictors.neural_network.neural_net_predictor import NeuralNetPredictor
from prescriptors.heuristics.heuristics import EvenHeuristic

In [2]:
dataset = ELUCData(start_year=2020, test_year=2021, countries=["US"])

Using the latest cached version of the dataset since projectresilience/ELUC-committed couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/964840/.cache/huggingface/datasets/projectresilience___ELUC-committed/default/0.0.0/c5965fb027a710b57d30c88f25c2ff627c05c011 (last modified on Fri Feb 16 15:28:56 2024).


In [3]:
from pathlib import Path

In [4]:
nn_config = {
    "features": constants.NN_FEATS,
    "label": "ELUC",
    "hidden_sizes": [4096],
    "linear_skip": True,
    "dropout": 0,
    "device": "mps",
    "epochs": 3,
    "batch_size": 2048,
    "train_pct": 1,
    "step_lr_params": {"step_size": 1, "gamma": 0.1},
}
nnp = NeuralNetPredictor(*nn_config)
nnp.load(Path("predictors/neural_network/trained_models/no_overlap_nn"))

In [5]:
heuristics = []
for pct in range(10, 110, 10):
    heuristics.append(EvenHeuristic(pct / 100, "secdf", nnp))

In [6]:
elucs = []
changes = []
for heuristic in heuristics:
    context_actions_df = heuristic.prescribe_land_use(None, None, dataset.test_df)
    eluc_df, change_df = heuristic.predict_metrics(context_actions_df)
    elucs.append(eluc_df["ELUC"].mean())
    changes.append(change_df["change"].mean())

type: The feature names should match those that were passed during fit.
Feature names must be in the same order as they were in fit.


In [ ]:
import matplotlib.pyplot as plt
plt.scatter(changes, elucs)